task name: LbfgsPoissonRegression
task description: Implement the L-BFGS Poisson Regression algorithm using ML.NET

In [ ]:
#r "nuget:Microsoft.ML,1.6.0"

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;

In [ ]:
var mlContext = new MLContext(seed: 0);

In [ ]:
// Example with label and 50 feature values. A data set is a collection of such examples.
private class DataPoint
{
    public float Label { get; set; }
    [VectorType(50)]
    public float[] Features { get; set; }
}

private static IEnumerable<DataPoint> GenerateRandomDataPoints(int count,
            int seed = 0)
{
    var random = new Random(seed);
    for (int i = 0; i < count; i++)
    {
        float label = (float)random.NextDouble();
        yield return new DataPoint
        {
            Label = label,
            // Create random features that are correlated with the label.
            Features = Enumerable.Repeat(label, 50).Select(
                x => x + (float)random.NextDouble()).ToArray()
        };
    }
}

var dataPoints = GenerateRandomDataPoints(1000).ToList();
var trainingData = mlContext.Data.LoadFromEnumerable(dataPoints);

In [ ]:
var pipeline = mlContext.Regression.Trainers.LbfgsPoissonRegression(
                labelColumnName: nameof(DataPoint.Label),
                featureColumnName: nameof(DataPoint.Features));

In [ ]:
var model = pipeline.Fit(trainingData);

In [ ]:
var testData = mlContext.Data.LoadFromEnumerable(GenerateRandomDataPoints(5, seed: 123));
var transformedTestData = model.Transform(testData);

In [ ]:
// Class used to capture predictions.
private class Prediction
{
    // Original label.
    public float Label { get; set; }
    // Predicted score from the trainer.
    public float Score { get; set; }
}

var predictions = mlContext.Data.CreateEnumerable<Prediction>(
                transformedTestData, reuseRowObject: false).ToList();

In [ ]:
foreach (var p in predictions)
{
    Console.WriteLine($"Label: {p.Label:F3}, Prediction: {p.Score:F3}");
}

In [ ]:
private static void PrintMetrics(RegressionMetrics metrics)
{
    Console.WriteLine($"Mean Absolute Error: {metrics.MeanAbsoluteError:F3}");
    Console.WriteLine($"Mean Squared Error: {metrics.MeanSquaredError:F3}");
    Console.WriteLine($"Root Mean Squared Error: {metrics.RootMeanSquaredError:F3}");
    Console.WriteLine($"RSquared: {metrics.RSquared:F3}");
}

var metrics = mlContext.Regression.Evaluate(transformedTestData);
PrintMetrics(metrics);